In [1]:
from scipy.special import expit
from rbm import RBM
from sampler import VanillaSampler, PartitionedSampler
from trainer import VanillaTrainier
from performance import Result
import numpy as np
import datasets, performance, plotter, mnist, pickle, rbm, os, logging, sampler
from sklearn.linear_model import Perceptron

logger = logging.getLogger()
# Set the logging level to logging.DEBUG 
logger.setLevel(logging.INFO)


%matplotlib inline

In [2]:
# make one 2 bit image, both visibles on
v = np.ones(2)

# lets make our hiddens, 2 hidden units
h_a = np.zeros(2)
h_b = np.zeros(2)
h_a[0] = 1
h_a[1] = 0

h_b[0] = 0
h_b[1] = 1

# Set up our weights matrix (|h| , |v|), with perfect weights.
w_a = np.ones((2, 2))
w_b = np.ones((2, 2))
w_a[0][1] = -1
w_a[1][0] = -1
w_b[0][1] = -1
w_b[1][0] = -1

In [3]:
A = RBM(2,2,1)
B = RBM(2,2,1)
# manually set our RBM's
A.hidden = h_a
B.hidden = h_b
A.visible = v
B.visible = v
A.weights = w_a
B.weights = w_b
A.hidden_bias = 0.5
B.hidden_bias = 0.5
A.visible_bias = 0.5
B.visible_bias = 0.5

In [4]:
def count_matrix_for_runs(runs, model, visible):
    van_sampler = VanillaSampler(model)
    counts = {}
    for i in range(runs):

        vis = np.where(van_sampler.hidden_to_visible(van_sampler.visible_to_hidden(visible)) >= 0.5, 1, 0)
        key = np.array_str(vis)
        if key in counts:
            counts[key] = counts[key] + 1
        else:
            counts[key] = 0
    for key in counts:
        counts[key] = counts[key]/runs * 100
    
    return counts

In [5]:
# run vanilla sampling 10,000 times, getting the percentage of reconstructions for A and then B
print(count_matrix_for_runs(10000, A, v))
print(count_matrix_for_runs(10000, B, v))

{'[1 0]': 23.46, '[1 1]': 52.43, '[0 1]': 24.08}
{'[1 0]': 23.810000000000002, '[1 1]': 53.410000000000004, '[0 1]': 22.75}


In [5]:
def weighted_sum_into_vis(vis_idx, w,h):
    return np.dot(w.T, h)

def visible_to_hidden(visible):
    np.dot(w.T,visibe)


In [6]:
def __bernouli_flip__(weighted_sum):
        p = expit(weighted_sum) > np.random.rand(*weighted_sum.shape)
        return np.where(p, 1, 0)
    
def calc_correction(hidden_a, hidden_b, weights_a, weights_b):        
    phi_a = np.dot(hidden_a, weights_a)
    phi_b = np.dot(hidden_b, weights_b)

    on_weights_a = (hidden_a * weights_a)
    off_weights_a = (1 - hidden_a) * weights_a

    on_weights_b =  (hidden_b * weights_b)
    off_weights_b = (1 - hidden_b) * weights_b
    
    j_off_a = phi_a - on_weights_a
    j_off_b = phi_b - on_weights_b
    j_on_a = phi_a + off_weights_a
    j_on_b = phi_b + off_weights_b
    
    correction_a = np.log(expit(j_off_a))  - np.log(expit((j_off_a + phi_b))) + np.log(expit((j_on_a + phi_b))) - np.log(expit(j_on_a))
    correction_b = np.log(expit(j_off_b))  - np.log(expit((j_off_b + phi_a))) + np.log(expit((j_on_b + phi_a))) - np.log(expit(j_on_b))
    #print("calc'd\nc_a{}\t for h_a{}\nc_b{}\t for h_b{}".format(correction_a, h_a, correction_b, h_b))
    return correction_a, correction_b



def run_partitioned_samples(hidden_a, hidden_b, weights_a, weights_b, visible,num_samples, vis_bias_a, vis_bias_b, hid_bias_a, hid_bias_b):
    for epoch in range(num_samples):

        phi_a = np.dot(hidden_a, weights_a) + vis_bias_a
        phi_b = np.dot(hidden_b, weights_b) + vis_bias_b
        
        correction_a, correction_b = calc_correction(hidden_a, hidden_b, weights_a, weights_b)
        """
        Apply the correction to the weighted sum into the hiddens
        """
        psi_a = (visible * weights_a.T + correction_a).sum(1) + hid_bias_a
        psi_b = (visible * weights_b.T + correction_b).sum(1) + hid_bias_b

        # now, do we turn on he hiddens? Bernoulli sample to decide
        hidden_a = __bernouli_flip__(psi_a)
        hidden_b = __bernouli_flip__(psi_b)

    return hidden_a, hidden_b

def runit(times, sample_times,h_a, h_b, w_a, w_b,v):
    wins = []
    for i in range(times):
        h_a, h_b = run_partitioned_samples(h_a,h_b, w_a,w_b, v, sample_times, 0, 0,0,0)
        wins.append((h_a != h_b).all())
        
    return wins

Recall that $$ \psi_{j} = \sum_i W_{ji} (v_i  + \sigma_{ij}^A - \sigma_{ij}^{AB}) $$
And therefore we would expect:
$$ \psi_{j=0} = \sum_i W_{0i} (v_i  + \sigma_{i0}^A - \sigma_{i0}^{AB}) $$

In [7]:
h_a, h_b = run_partitioned_samples(h_a,h_b, w_a,w_b, v, 500, 0.5, 0.5,0.5,0.5)
print("h_a{} h_b{}".format(h_a, h_b))

h_a[1 1] h_b[1 1]


In [8]:
# do 200 runs of partitioned sampling with each run finding the hidden state with 10,000 gibbs alternations
wins = runit(200,10000, h_a, h_b, w_a,w_b,v)
win_count = np.array(wins).sum()
lose_count = len(wins) - win_count
print("wins:{} losses:{}".format(win_count, lose_count))
# damn we wanted to wins to be greater than the losses.

wins:57 losses:143


In [104]:
c_a, c_b = calc_correction(np.array([1,1]), np.array([0,1]), w_a, w_b)

$$ \psi_{j} = \sum_{i} (W_{ji}v_i + C_{ji}) $$

In [126]:
def approx_correction(h_a, h_b, w_a, w_b, v_bias, h_bias):
    phi_a = np.dot(w_a, h_a)
    phi_b = np.dot(w_b, h_b)
    sig_A = phi_a + w_a/2
    sig_B = phi_b + w_b/2
    epsilon_a = np.dot(w_b,h_b)
    epsilon_b = np.dot(w_a,h_a)
    sig_AB = sig_A + epsilon_a
    sig_BA = sig_B + epsilon_b
    c_a = expit(sig_A) - expit(sig_AB)
    c_b = expit(sig_B) - expit(sig_BA)
    return c_a, c_b

def psi(h_a, h_b, w_a, w_b, v_bias = 0.5, h_bias = 0.5):
    c_a, c_b = approx_correction(h_a, h_b, w_a, w_b, v_bias, h_bias)
    psi_a = (w_a * (v + c_a)).sum(1)
    psi_b = (w_b * (v + c_b)).sum(1)
    return psi_a,psi_b

In [127]:
psi(np.array([1,0]), np.array([0,1]), w_a, w_b)

(array([ 0.39023029, -0.48983732]), array([-0.48983732,  0.39023029]))

In [128]:
psi(np.array([1,1]), np.array([0,1]), w_a, w_b)

(array([ 0.48983732, -0.39023029]), array([ 0.,  0.]))

In [129]:
psi(np.array([0,0]), np.array([0,1]), w_a, w_b)

(array([ 0.48983732, -0.39023029]), array([ 0.,  0.]))

In [130]:
psi(np.array([1,0]), np.array([1,1]), w_a, w_b)

(array([ 0.,  0.]), array([-0.39023029,  0.48983732]))

In [131]:
psi(np.array([1,0]), np.array([1,0]), w_a, w_b)

(array([-0.21313469,  0.39023029]), array([-0.21313469,  0.39023029]))

In [132]:
psi(np.array([0,1]), np.array([0,1]), w_a, w_b)

(array([ 0.39023029, -0.21313469]), array([ 0.39023029, -0.21313469]))

In [133]:
psi(np.array([1,1]), np.array([1,1]), w_a, w_b)

(array([ 0.,  0.]), array([ 0.,  0.]))

In [134]:
psi(np.array([0,0]), np.array([0,0]), w_a, w_b)

(array([ 0.,  0.]), array([ 0.,  0.]))